# preprocess

In [ ]:
!pip install openai
import pandas as pd
import plotly.express as px

!pip install dtreeviz
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
import dtreeviz

import numpy as np
import statsmodels.api as sm

In [ ]:
df=pd.read_excel('/content/drive/MyDrive/Ideas/to job/case/Informe Alumnos Insurgentes - 12_04_2024 (2).xlsx')
# df.to_csv('/content/drive/MyDrive/Ideas/to job/case/Informe Alumnos Insurgentes Processed.csv',index=False)
df

,Email,Nombre y Apellido,Institucion,Año,Curso,Probabilidad UIN,Carrera 1,Carrera 2,Carrera 3,Universidad 1,Universidad 2,Universidad 3
0,acxel.aco@uinenlinea.mx,ACXEL HAZIEL ACO GIJON,Insurgentes Chalco,6,UNAM,1. Alta,ingenieria aeronaval (piloto),infanteria de la marina,rally,UIN,ejercito mexicano,NaN
1,alejandro.juarez.m@uinenlinea.mx,ALEJANDRO JUAREZ MEDRANO,Insurgentes Chalco,6,UNAM,3. Baja,NaN,NaN,NaN,IBERO,UNAM,NaN
2,alvaro.alvarez@uinenlinea.mx,ALVARO YAEL ALVAREZ RODRIGUEZ,Insurgentes Chalco,6,UNAM,3. Baja,Fisioterapia,Quiropractica,Nutricion,UNITEC,IPET,IPN
3,ana.fernandez.m@uinenlinea.mx,ANA KAREN FERNANDEZ MELENDEZ,Insurgentes Chalco,6,UNAM,3. Baja,Medico Veterinaria Zooctenista,Químico Farmacéutico Industrial,Finanzas y Banca,UNAM,UAM,Politecnico
4,andres.martinez.v@uinenlinea.mx,ANDRES MARTINEZ VILLEGAS,Insurgentes Chalco,6,UNAM,4.No sabe/No contesta,contaduria,psicologia,arquitecto,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
501,omar.chimal@uinenlinea.mx,OMAR CHIMAL BERNARDO,Insurgentes Toreo,6,UIN,3. Baja,Ingeniería en sistemas computacionales,Arquitecto,Chef,Unam,Politécnico,Uam
502,ricardo.fernandez@uinenlinea.mx,RICARDO ARIEL FERNANDEZ GONZALEZ,Insurgentes Toreo,6,UIN,3. Baja,Fisica,Ingeniería en Inteligencia Artificial,Biología,UNAM,Politécnico,Tecnológico de Monterrey
503,rodrigo.lopez.p@uinenlinea.mx,RODRIGO LOPEZ PEREZ,Insurgentes Toreo,6,UIN,4.No sabe/No contesta,NaN,NaN,NaN,NaN,NaN,NaN
504,samuel.garcia.b@uinenlinea.mx,SAMUEL GARCIA BARRIENTOS,Insurgentes Toreo,6,UIN,1. Alta,Psicología,Psicología,Psicología,Uin,Heroico militar,Uvm


In [ ]:
len(df)

In [ ]:

from pydantic import BaseModel
from openai import OpenAI
from pydantic import conlist


def get_carrer(carrers):
    client = OpenAI(api_key=openai_key)
    class InputCarrer(BaseModel):
      carrer: str
      clusteredCarrer: str

    class MapOfCarrers(BaseModel):
        simplifiedCareers: list[InputCarrer] #list[list[str]] #conlist(str, min_length=506, max_length=506) #

    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": '''I will give you a list of careers that diferent people want to follow, but the thing is that there are too many cases,
            so i will like you to cluster it in at most 10 categories,
            and then give me a dict that maps the results,so for example if
            the input is 'Ingeniería en Inteligencia Artificial' then the output should be carrer='Ingeniería en Inteligencia Artificial' and  clusteredCarrer='Ingenieria']'''},
            {"role": "user", "content": carrers},
        ],
        response_format=MapOfCarrers,
    )

    result = completion.choices[0].message.parsed.simplifiedCareers
    return result

# clustered_careers=get_carrer(str(df['Carrera 1'].tolist()))
clustered_careers=get_carrer(str(list(set(df['Carrera 1'].tolist()))))
mapper={}
for carrer,clusteredCarrer in clustered_careers:
  mapper[carrer[1]]=clusteredCarrer[1]
mapper

{'medicina': 'Medicina',
 'Administración': 'Administración',
 'Diseño de Modas': 'Diseño',
 'Veterinaria': 'Veterinaria',
 'ingeniera en sistemas': 'Ingeniería',
 'fisioterapia': 'Salud',
 'Ingeniería en tecnologías de la comunicación': 'Ingeniería',
 'Turismo': 'Turismo',
 'Enfermeria': 'Salud',
 'DERECHO': 'Derecho',
 'Fisica': 'Ciencias Exactas',
 'Ingenierías de automatización': 'Ingeniería',
 'Criminalística': 'Ciencias Forenses',
 'ARQUITECTURA': 'Arquitectura',
 'Enfermería general': 'Salud',
 'medico': 'Medicina',
 'Ingeniería mecánica': 'Ingeniería',
 'Enfermería y obstetricia .': 'Salud',
 'psicología': 'Psicología',
 'Medico Veterinaria Zooctenista': 'Veterinaria',
 'Producción musical': 'Artes y Música',
 'Imaginería Civil': 'Ingeniería',
 'odontologia': 'Odontología',
 'Contaduria': 'Contabilidad',
 'ODONTOLOGIA': 'Odontología',
 'Negocios Internacionales': 'Negocios',
 'Licenciatura en Turismo': 'Turismo',
 'Composición musical': 'Artes y Música',
 'contabilidad': 'Conta

In [ ]:
def get_gender(names):
    client = OpenAI(api_key=openai_key)
    class InputName(BaseModel):
      name: str
      gender: str

    class MapOfGender(BaseModel):
        map: list[InputName] #list[list[str]] #conlist(str, min_length=506, max_length=506) #

    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": '''I will give you a list of names, and i want you to return a map of the name of the person and its probable gender, keep the input name intact'''},
            {"role": "user", "content": names},
        ],
        response_format=MapOfGender,
    )

    result = completion.choices[0].message.parsed.map
    return result
genders=get_gender(str(list(set(df['Nombre y Apellido'].tolist()))))
gmapper={}
for carrer,clusteredCarrer in genders:
  gmapper[carrer[1]]=clusteredCarrer[1]
gmapper


{'ORLANDO CARDENAS MARTINEZ': 'Male',
 'DIEGO ALEJANDRO BALBOA MELO': 'Male',
 'REGINA PAOLA FLORES GALLARDO': 'Female',
 'CARLOS GAEL ROJAS RUELA': 'Male',
 'SANTIAGO CARBALLEDA SIERRA': 'Male',
 'ARANZA AMELLALLI BARRUETA GONZALEZ': 'Female',
 'NANCY ELISUA CASTRO PANTALEON': 'Female',
 'FRANCISCO JAVIER PEREZ RIVERA': 'Male',
 'DULCE MIA GOMEZ MERINO': 'Female',
 'LEONARDO MADRIGAL NIETO': 'Male',
 'ALEJANDRO JUAREZ MEDRANO': 'Male',
 'REBECA ANDREA SANCHEZ RODRIGUEZ': 'Female',
 'SARAHI PATRICIO MARURI': 'Female',
 'LISSETT YOANI HERNANDEZ RAMIREZ': 'Female',
 'MARYJOSE MERLIN ORTEGA': 'Female',
 'MELISSA CAMPOS CARRASCO': 'Female',
 'REGINA DAHELY AGUILAR CARREÑO': 'Female',
 'OMAR CHIMAL BERNARDO': 'Male',
 'GAEL ARTURO GARCIA GRANADOS': 'Male',
 'KIARA GUADALUPE SAMANO RESENDIZ': 'Female',
 'AZUL ARELI TORRES ZEPEDA': 'Female',
 'AZUL MARINA ACOSTA RAZO': 'Female',
 'CRISTIAN GUERRERO VARGAS': 'Male',
 'VALERIA BRITO MENDOZA': 'Female',
 'DEREK ABDIEL GARCIA LARA': 'Male',
 'SEL

In [ ]:
df['Carrera 1 Clustered']=df['Carrera 1'].apply(lambda x: mapper[x] if x in mapper.keys() else 'none')
df['Gender']=df['Nombre y Apellido'].apply(lambda x: gmapper[x] if x in gmapper.keys() else 'none')
df['University']=df['Universidad 1'].apply(lambda x: 'own' if x in ['UIN','UNAM'] else 'other')
df['y']=df['Universidad 1'].apply(lambda x: 1. if x in ['UIN','UNAM'] else 0.)
df

,Email,Nombre y Apellido,Institucion,Año,Curso,Probabilidad UIN,Carrera 1,Universidad 1,Carrera 1 Clustered,University,Gender,y
0,acxel.aco@uinenlinea.mx,ACXEL HAZIEL ACO GIJON,Insurgentes Chalco,6,UNAM,1. Alta,ingenieria aeronaval (piloto),UIN,Aviación,own,Male,1.0
1,alejandro.juarez.m@uinenlinea.mx,ALEJANDRO JUAREZ MEDRANO,Insurgentes Chalco,6,UNAM,3. Baja,NaN,IBERO,none,other,Male,0.0
2,alvaro.alvarez@uinenlinea.mx,ALVARO YAEL ALVAREZ RODRIGUEZ,Insurgentes Chalco,6,UNAM,3. Baja,Fisioterapia,UNITEC,Salud,other,Male,0.0
3,ana.fernandez.m@uinenlinea.mx,ANA KAREN FERNANDEZ MELENDEZ,Insurgentes Chalco,6,UNAM,3. Baja,Medico Veterinaria Zooctenista,UNAM,Veterinaria,own,Female,1.0
4,andres.martinez.v@uinenlinea.mx,ANDRES MARTINEZ VILLEGAS,Insurgentes Chalco,6,UNAM,4.No sabe/No contesta,contaduria,NaN,Contabilidad,other,Male,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
501,omar.chimal@uinenlinea.mx,OMAR CHIMAL BERNARDO,Insurgentes Toreo,6,UIN,3. Baja,Ingeniería en sistemas computacionales,Unam,Ingeniería,other,Male,0.0
502,ricardo.fernandez@uinenlinea.mx,RICARDO ARIEL FERNANDEZ GONZALEZ,Insurgentes Toreo,6,UIN,3. Baja,Fisica,UNAM,Ciencias Exactas,own,Male,1.0
503,rodrigo.lopez.p@uinenlinea.mx,RODRIGO LOPEZ PEREZ,Insurgentes Toreo,6,UIN,4.No sabe/No contesta,NaN,NaN,none,other,Male,0.0
504,samuel.garcia.b@uinenlinea.mx,SAMUEL GARCIA BARRIENTOS,Insurgentes Toreo,6,UIN,1. Alta,Psicología,Uin,Psicología,other,Male,0.0


In [ ]:

second_map={'none':'None',
'Salud':'Medicine', 'Veterinaria':'Medicine','Medicina':'Medicine','Odontología':'Medicine','Ciencias Forenses':'Medicine',
'Psicología':'Psychology',
'Contabilidad':'Economics','Administración':'Economics', 'Negocios':'Economics','Economía':'Economics',
'Aviación':'Other',
'Gastronomía':'Gastronomy',
'Ingeniería':'Engineering',
'Diseño':'Arts','Artes y Música':'Arts',
'Ciencias Exactas':'STEM',
'Arquitectura':'Architecture',
'Derecho':'Law',
'Comunicación':'Social Sciences', 'Información y Comunicación':'Social Sciences','Ciencias Sociales':'Social Sciences',
'Educación':'Education',
'Turismo':'Other',
'Cosmetología':'Other',
}
df['Carrera 1 Clustered']=df['Carrera 1 Clustered'].apply(lambda x:second_map[x])
df.value_counts('Carrera 1 Clustered')

<ipython-input-90-2a5566fb06b8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,count
Carrera 1 Clustered,
None,191
Medicine,60
Economics,45
Engineering,41
Arts,38
Law,29
Psychology,23
Architecture,19
Other,16


In [ ]:
df=df[['Email', 'Nombre y Apellido', 'Institucion', 'Año', 'Curso',
       'Probabilidad UIN', 'Carrera 1',
       'Universidad 1',
       'Carrera 1 Clustered', 'University','y','Gender']]
df

,Email,Nombre y Apellido,Institucion,Año,Curso,Probabilidad UIN,Carrera 1,Universidad 1,Carrera 1 Clustered,University,y,Gender
0,acxel.aco@uinenlinea.mx,ACXEL HAZIEL ACO GIJON,Insurgentes Chalco,6,UNAM,1. Alta,ingenieria aeronaval (piloto),UIN,Other,own,1.0,Male
1,alejandro.juarez.m@uinenlinea.mx,ALEJANDRO JUAREZ MEDRANO,Insurgentes Chalco,6,UNAM,3. Baja,NaN,IBERO,None,other,0.0,Male
2,alvaro.alvarez@uinenlinea.mx,ALVARO YAEL ALVAREZ RODRIGUEZ,Insurgentes Chalco,6,UNAM,3. Baja,Fisioterapia,UNITEC,Medicine,other,0.0,Male
3,ana.fernandez.m@uinenlinea.mx,ANA KAREN FERNANDEZ MELENDEZ,Insurgentes Chalco,6,UNAM,3. Baja,Medico Veterinaria Zooctenista,UNAM,Medicine,own,1.0,Female
4,andres.martinez.v@uinenlinea.mx,ANDRES MARTINEZ VILLEGAS,Insurgentes Chalco,6,UNAM,4.No sabe/No contesta,contaduria,NaN,Economics,other,0.0,Male
...,...,...,...,...,...,...,...,...,...,...,...,...
501,omar.chimal@uinenlinea.mx,OMAR CHIMAL BERNARDO,Insurgentes Toreo,6,UIN,3. Baja,Ingeniería en sistemas computacionales,Unam,Engineering,other,0.0,Male
502,ricardo.fernandez@uinenlinea.mx,RICARDO ARIEL FERNANDEZ GONZALEZ,Insurgentes Toreo,6,UIN,3. Baja,Fisica,UNAM,STEM,own,1.0,Male
503,rodrigo.lopez.p@uinenlinea.mx,RODRIGO LOPEZ PEREZ,Insurgentes Toreo,6,UIN,4.No sabe/No contesta,NaN,NaN,None,other,0.0,Male
504,samuel.garcia.b@uinenlinea.mx,SAMUEL GARCIA BARRIENTOS,Insurgentes Toreo,6,UIN,1. Alta,Psicología,Uin,Psychology,other,0.0,Male


In [ ]:
# df.to_csv('/content/drive/MyDrive/Ideas/to job/case/Informe Alumnos Insurgentes Processed.csv',index=False)

# graphs

In [ ]:
#bar
var='Carrera 1 Clustered'

df['ones']=1.
color='University'
px.bar(df.groupby([var,color],as_index=False).sum().sort_values('ones',ascending=False),x=var,y='ones',color=color).show()
px.bar(df.groupby([var],as_index=False).mean(numeric_only=True).sort_values('y',ascending=False),x=var,y='y').show()
px.scatter(df.groupby([var],as_index=False).agg({'y':'mean','ones':'sum'}),x='ones',y='y',color=var,log_x=True).show()

<ipython-input-170-ba32d1dd7964>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df['Carrera 1 Clustered'].unique()

array(['Aviación', 'none', 'Salud', 'Veterinaria', 'Contabilidad',
       'Gastronomía', 'Ingeniería', 'Medicina', 'Diseño',
       'Artes y Música', 'Negocios', 'Ciencias Exactas', 'Arquitectura',
       'Administración', 'Derecho', 'Odontología', 'Psicología',
       'Comunicación', 'Ciencias Forenses', 'Educación', 'Turismo',
       'Cosmetología', 'Información y Comunicación', 'Economía',
       'Ciencias Sociales'], dtype=object)

In [ ]:
# Carrera 1
#exactas ingenieria paradox
#nan puede ser leak
df[df['Carrera 1 Clustered']=='Información y Comunicación']
df[df['Carrera 1 Clustered']=='none']

#anio 6 double almost
#probabilidad es leaked

,Email,Nombre y Apellido,Institucion,Año,Curso,Probabilidad UIN,Carrera 1,Universidad 1,Carrera 1 Clustered,University,y,Gender,ones
1,alejandro.juarez.m@uinenlinea.mx,ALEJANDRO JUAREZ MEDRANO,Insurgentes Chalco,6,UNAM,3. Baja,NaN,IBERO,none,other,0.0,Male,1.0
7,carmen.gonzalez.j@uinenlinea.mx,CARMEN FABIOLA GONZALEZ JUAREZ,Insurgentes Chalco,6,UNAM,3. Baja,NaN,Escuela de Aviación México,none,other,0.0,Female,1.0
11,deivit.lopezportillo@uinenlinea.mx,DEIVIT ALEXIS LOPEZ PORTILLO VALADEZ,Insurgentes Chalco,6,UNAM,4.No sabe/No contesta,NaN,NaN,none,other,0.0,Male,1.0
18,eduardo.hernandez.m@uinenlinea.mx,EDUARDO HERNANDEZ MORENO,Insurgentes Chalco,6,UNAM,4.No sabe/No contesta,NaN,NaN,none,other,0.0,Male,1.0
22,german.jardon@uinenlinea.mx,GERMAN ABDIEL JARDON VEGA,Insurgentes Chalco,6,UNAM,1. Alta,NaN,UIN,none,own,1.0,Male,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,yoab.limon@uinenlinea.mx,YOAB JESUS LIMON CASASOLA,Insurgentes Tlalpan,6,UNAM,3. Baja,NaN,UNAM,none,own,1.0,Male,1.0
491,kevin.garcia.p@uinenlinea.mx,KEVIN GUILLERMO GARCIA POSADAS,Insurgentes Toreo,6,UIN,3. Baja,NaN,UVM,none,other,0.0,Male,1.0
492,kevin.romero.h@uinenlinea.mx,KEVIN ROLANDO ROMERO HERNANDEZ,Insurgentes Toreo,6,UIN,4.No sabe/No contesta,NaN,NaN,none,other,0.0,Male,1.0
496,luis.morales.martinez@uinenlinea.mx,LUIS ANGEL MORALES MARTINEZ,Insurgentes Toreo,6,UIN,4.No sabe/No contesta,NaN,NaN,none,other,0.0,Male,1.0


In [ ]:
df['Carrera 1 Clustered'].unique()

array(['Aviación', 'none', 'Salud', 'Veterinaria', 'Contabilidad',
       'Gastronomía', 'Ingeniería', 'Medicina', 'Diseño',
       'Artes y Música', 'Negocios', 'Ciencias Exactas', 'Arquitectura',
       'Administración', 'Derecho', 'Odontología', 'Psicología',
       'Comunicación', 'Ciencias Forenses', 'Educación', 'Turismo',
       'Cosmetología', 'Información y Comunicación', 'Economía',
       'Ciencias Sociales'], dtype=object)

# Sankey

In [ ]:
import plotly.graph_objects as go
l=[px.colors.qualitative.Alphabet for i in range(100)]
colores=px.colors.qualitative.Plotly+px.colors.qualitative.D3+px.colors.qualitative.G10+px.colors.qualitative.T10+px.colors.qualitative.Alphabet+[item for sublist in l for item in sublist]

def sankey(df_pandas,categorical_dimensions,ys,height,width,colorOn):
  first_group=df_pandas.groupby(categorical_dimensions).sum().reset_index()
  rowQuantity=1000
  first_group["q"]=np.floor(rowQuantity*(first_group[ys]/first_group[ys].sum()))
  first_group["color"]=0
  counter=0
  for i in first_group[categorical_dimensions[colorOn]]:
    #first_group.loc[first_group[categorical_dimensions[-1]]==i,"color"]=counter/len(first_group[categorical_dimensions[-1]].unique())
    first_group.loc[first_group[categorical_dimensions[colorOn]]==i,"color"]=colores[counter]
    counter=counter+1

  first_group=first_group.loc[first_group.index.repeat(first_group.q)]
  dimensions = [dict(values=first_group[label], label=label) for label in categorical_dimensions]
  # Build colorscale
  #color = np.zeros(len(first_group), dtype='uint8')

  color = first_group["color"].tolist()

  colorscale = [[0, 'gray'], [1, 'firebrick']]

  # Build figure as FigureWidget
  fig = go.FigureWidget(
      data=[ go.Parcats(
          # domain={'y': [0, 0.4]},
          dimensions=dimensions,
          line={
              # 'colorscale': colorscale,
              # 'cmin': 0,
              #   'cmax': 1,
                'color': color,
                'shape': 'hspline'})
      ])

  fig.update_layout(

          height=height,
          width=width,
          margin={'t':20,'l':20,'b':20,'r':0},
          # xaxis={'title': 'Horsepower'},
          # yaxis={'title': 'MPG', 'domain': [0.6,1]},
          dragmode='lasso', hovermode='closest')



  fig.show()
sankey(df,x_cols+['University'],'ones',500,1000,-1)

<ipython-input-217-207b6e670733>:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '#636EFA' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.


# linear reg

In [ ]:
x_cols=['Gender','Año','Institucion','Curso','Carrera 1 Clustered']
X=pd.get_dummies(df[x_cols], dtype='float').to_numpy()
X = sm.add_constant(X)
y=df['y'].to_numpy()
X

array([[1., 6., 0., ..., 0., 0., 0.],
       [1., 6., 0., ..., 0., 0., 0.],
       [1., 6., 0., ..., 0., 0., 0.],
       ...,
       [1., 6., 0., ..., 0., 0., 0.],
       [1., 6., 0., ..., 1., 0., 0.],
       [1., 6., 1., ..., 0., 0., 0.]])

In [ ]:


results = sm.OLS(y, X).fit()
print(results.summary(xname=['constant']+list(pd.get_dummies(df[x_cols], dtype='float').columns)))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.142
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     3.058
Date:                Tue, 24 Sep 2024   Prob (F-statistic):           1.06e-06
Time:                        13:48:04   Log-Likelihood:                -181.28
No. Observations:                 506   AIC:                             416.6
Df Residuals:                     479   BIC:                             530.7
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

# logit

In [ ]:
x_cols=['Gender','Año','Institucion','Curso','Carrera 1 Clustered']
exclude=['Gender_Male','Institucion_Insurgentes León','Curso_BIV EN INF ADM','Carrera 1 Clustered_Law']
meta_df=pd.get_dummies(df[x_cols], dtype='float')
no_corr=[]
for j in x_cols:
  no_corr=no_corr+ [i for i in meta_df.columns if (j in i) and (i not in exclude)]


In [ ]:

X=meta_df[no_corr].to_numpy()

y=df['y'].to_numpy()

results = sm.Logit(y, X).fit()
print(results.summary(xname=no_corr))

Optimization terminated successfully.
         Current function value: 0.379998
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  506
Model:                          Logit   Df Residuals:                      480
Method:                           MLE   Df Model:                           25
Date:                Tue, 24 Sep 2024   Pseudo R-squ.:                  0.1606
Time:                        13:51:52   Log-Likelihood:                -192.28
converged:                       True   LL-Null:                       -229.05
Covariance Type:            nonrobust   LLR p-value:                 1.128e-06
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Gender_Female                          -0.1086      0.282 

# tree

In [ ]:
x_cols=['Institucion','Año','Curso','Carrera 1 Clustered','Gender']

# meta_df=df.copy()
# for i in x_cols:
#   meta_df[i]=pd.factorize(meta_df[i])[0]
meta_df=pd.get_dummies(df[x_cols], dtype='float')
x_cols_2=list(meta_df.columns)
meta_df['y']=df['y'].astype(int)

In [ ]:
# %%capture
# iris = load_iris()

X_meta = meta_df[x_cols_2].to_numpy()
y_meta = meta_df['y'].to_numpy()

# clf = DecisionTreeClassifier(max_depth=5)
#first 1e-3
nodes=[]
space=np.logspace(0,-5,num=1000)
for cut in space:
  clf = DecisionTreeClassifier(min_impurity_decrease=cut,min_samples_leaf=10)
  clf.fit(X_meta, y_meta)
  nodes.append(clf.tree_.node_count)

px.line(x=space,y=nodes,log_x=True).show()
px.scatter(x=space[1:],y=np.diff(nodes),log_x=True).show()


In [ ]:
try:
  cutter=space[1:][np.argmax(np.diff(nodes))-1]+1e-10
except:
  cutter=1e-10
clf = DecisionTreeClassifier(min_impurity_decrease=cutter,min_samples_leaf=10)
clf.fit(X_meta, y_meta)
nodes.append(clf.tree_.node_count)
viz_model = dtreeviz.model(clf,
                           X_train=X_meta, y_train=y_meta,
                           feature_names=x_cols_2,
                           target_name='University',
                           class_names=['other', 'own',])

v = viz_model.view()
# v.show()
# displayHTML(v.svg())
v.save("mini_pred.svg")